In [1]:
from pandas import read_csv
import pandas as pd
import numpy as np

data = pd.read_csv("clean_indonesia_data.csv")

In [2]:
del data['date']
del data['fully_vaccin']

In [3]:
dataset = pd.read_csv("mi.csv")
del dataset['Unnamed: 0']
dataset.columns = ['relative_change']

In [4]:
data = data.assign(rc = dataset['relative_change'])

In [5]:
from pandas import read_csv
from numpy import concatenate
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from keras.layers import Dropout

In [6]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [7]:
values = data.values
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [8]:
reframed = series_to_supervised(scaled, 7, 7)

In [9]:
for i in range (22,29,1):
    reframed.drop(reframed.columns[[i,i+1]], axis=1, inplace=True)

reframed

,var1(t-7),var2(t-7),var3(t-7),var1(t-6),var2(t-6),var3(t-6),var1(t-5),var2(t-5),var3(t-5),var1(t-4),...,var1(t-1),var2(t-1),var3(t-1),var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6)
7,0.000035,0.000000,0.899841,0.000000,0.000000,0.902921,0.000000,0.000000,0.872913,0.000000,...,0.000035,0.000000,0.904446,0.000229,0.000141,0.000123,0.000000,0.000617,0.000476,0.000370
8,0.000000,0.000000,0.902921,0.000000,0.000000,0.872913,0.000000,0.000000,0.882225,0.000035,...,0.000229,0.000000,0.939527,0.000141,0.000123,0.000000,0.000617,0.000476,0.000370,0.000300
9,0.000000,0.000000,0.872913,0.000000,0.000000,0.882225,0.000035,0.000000,0.901183,0.000000,...,0.000141,0.000000,0.936232,0.000123,0.000000,0.000617,0.000476,0.000370,0.000300,0.000670
10,0.000000,0.000000,0.882225,0.000035,0.000000,0.901183,0.000000,0.000000,0.923092,0.000035,...,0.000123,0.000000,0.920670,0.000000,0.000617,0.000476,0.000370,0.000300,0.000670,0.000969
11,0.000035,0.000000,0.901183,0.000000,0.000000,0.923092,0.000035,0.000000,0.904446,0.000229,...,0.000000,0.000000,0.909545,0.000617,0.000476,0.000370,0.000300,0.000670,0.000969,0.001480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,0.024385,0.865302,0.905285,0.020561,0.876119,0.933826,0.015751,0.883311,0.908175,0.010924,...,0.018553,0.919632,0.898434,0.016121,0.017566,0.013161,0.011029,0.015910,0.016104,0.011153
593,0.020561,0.876119,0.933826,0.015751,0.883311,0.908175,0.010924,0.884487,0.932771,0.022218,...,0.016121,0.929824,0.903313,0.017566,0.013161,0.011029,0.015910,0.016104,0.011153,0.013390
594,0.015751,0.883311,0.908175,0.010924,0.884487,0.932771,0.022218,0.893661,0.920169,0.021724,...,0.017566,0.940450,0.938707,0.013161,0.011029,0.015910,0.016104,0.011153,0.013390,0.014130
595,0.010924,0.884487,0.932771,0.022218,0.893661,0.920169,0.021724,0.905324,0.916101,0.018553,...,0.013161,0.947797,0.856616,0.011029,0.015910,0.016104,0.011153,0.013390,0.014130,0.010977


In [10]:
values = reframed.values

train = values[:-91, :]
test = values[-91:-1, :]

train_X, train_y = train[:,:-7], train[:,-7:]
test_X, test_y = test[:,:-7], test[:,-7:]

train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [11]:
def build_regressor(neurons=1,dropout_rate=0.01,
                   learn_rate=0.01, decays=0.01):
    grid_model = Sequential()
    grid_model.add(LSTM(units=neurons, activation='tanh',input_shape=(train_X.shape[1], train_X.shape[2])))
    grid_model.add(Dropout(dropout_rate))
    grid_model.add(Dense(7))
    
    #compile model
    optimizer=Adam(lr=learn_rate, decay=decays)
    grid_model.compile(loss='mae',  optimizer=optimizer)
    return grid_model

grid_model = KerasRegressor(build_fn=build_regressor, epochs=3000, 
                            batch_size = 30, verbose=0)
parameters = {'neurons' : [9, 3],
              'learn_rate' : [0.001,0.01, 0.1],
              'decays':[0.001,0.0001,0.01],
              'dropout_rate':[0.05,0.01]}

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters)

In [12]:
tes = grid_search.fit(train_X, train_y,
                      validation_data=(test_X, test_y))

C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\Achmad

C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\Achmad

C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\Achmad

C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\Achmad

C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\AchmadFai\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\Achmad

In [13]:
# summarize results
print("Best parameters: %f using %s\n" % (grid_search.best_score_, grid_search.best_params_))

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best parameters: -0.031327 using {'decays': 0.01, 'dropout_rate': 0.01, 'learn_rate': 0.001, 'neurons': 9}

-0.041497 (0.061542) with: {'decays': 0.001, 'dropout_rate': 0.05, 'learn_rate': 0.001, 'neurons': 9}
-0.043966 (0.066509) with: {'decays': 0.001, 'dropout_rate': 0.05, 'learn_rate': 0.001, 'neurons': 3}
-0.047914 (0.074033) with: {'decays': 0.001, 'dropout_rate': 0.05, 'learn_rate': 0.01, 'neurons': 9}
-0.047380 (0.074291) with: {'decays': 0.001, 'dropout_rate': 0.05, 'learn_rate': 0.01, 'neurons': 3}
-0.064982 (0.068328) with: {'decays': 0.001, 'dropout_rate': 0.05, 'learn_rate': 0.1, 'neurons': 9}
-0.062648 (0.065639) with: {'decays': 0.001, 'dropout_rate': 0.05, 'learn_rate': 0.1, 'neurons': 3}
-0.040706 (0.060037) with: {'decays': 0.001, 'dropout_rate': 0.01, 'learn_rate': 0.001, 'neurons': 9}
-0.041772 (0.060147) with: {'decays': 0.001, 'dropout_rate': 0.01, 'learn_rate': 0.001, 'neurons': 3}
-0.047590 (0.072487) with: {'decays': 0.001, 'dropout_rate': 0.01, 'learn_rate': 0